### Loading Libraries

In [18]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# EDA and Preprocessing

The purpose of this notebook is to explore the invoice dataset. Several cleaning methods are explored that are later summarized in the *preporocessing()* function in the utils.py file.

In [19]:
df = pd.read_csv('data/invoices_basic.csv')

## Removing Missing Values

In [ ]:
# Check for missing value in each column
missing_values = df.isnull().sum()
print(missing_values)
# Drop rows with missing values
df = df.dropna()

## Removing False Entries

- All entries before 2018 defined as false entries
- All rows were due_date < receipt_date false entries
- All rows were due-date = receipt_date excluded because no prediction is necessary on those invoices


In [ ]:
# Remove all entries where weighted_payment_date <= receipt_date
df = df[df['weighted_payment_date'] > df['receipt_date']]

rows_before_removing = df.shape[0]
len_before_removing = len(df)

#Remove all  entries where Receipt_date < 01.01.2018
df = df[df['receipt_date'] >= '2018-01-01']

# Check what % of rows have due_date < receipt_date
due_before_receipt = (df['due_date'] < df['receipt_date']).mean() * 100
print(f"Percentage of rows where due_date < receipt_date: {due_before_receipt:.2f}%")

# Check what % of rows have due_date == receipt_date
due_equals_receipt = (df['due_date'] == df['receipt_date']).mean() * 100
print(f"Percentage of rows where due_date == receipt_date: {due_equals_receipt:.2f}%")

# Remove all entries where due_date <= receipt_date
df = df[df['due_date'] > df['receipt_date']]

# Check % of rows where payment_terms == 0
payment_terms_zero = (df['payment_terms'] == 0).mean() * 100
print(f"Percentage of rows where payment_terms == 0: {payment_terms_zero:.2f}%")

# Sort the data by 'receipt_date' in ascending order
df = df.sort_values('receipt_date')
rows_after_removing = df.shape[0]
print(f"Number of rows before removing: {rows_before_removing}")
print(f"Number of rows after removing: {rows_after_removing}")
print(f"Percentage of rows after removing: {rows_after_removing/len_before_removing:.2f}%")

## Amount of A/Rs over time

Plotting amount of entries over time to decide which timespan to include in dataset for ML Model. Graphs created for "invoices", "reminders" and "clarifications"

Timespan: 01.01.2022 - 01.08.2024

### Invoice Dataset:

In [ ]:
# Convert 'receipt_date' and 'weighted_payment_date' to datetime format if they are not already
df['receipt_date'] = pd.to_datetime(df['receipt_date'])
df['weighted_payment_date'] = pd.to_datetime(df['weighted_payment_date'])
df['due_date'] = pd.to_datetime(df['due_date'])

# Create copies of the dataframe
graph_receipt = df.copy()
graph_payment = df.copy()

# Set 'receipt_date' and 'weighted_payment_date' as the index
graph_receipt.set_index('receipt_date', inplace=True)
graph_payment.set_index('weighted_payment_date', inplace=True)

# Resample the data to monthly frequency and count the number of rows for each month
monthly_receipt_counts = graph_receipt.resample('ME').size().reset_index(name='receipt_count')
monthly_payment_counts = graph_payment.resample('ME').size().reset_index(name='payment_count')

# Merge the two DataFrames on the 'receipt_date' column
monthly_counts = pd.merge(monthly_receipt_counts, monthly_payment_counts, left_on='receipt_date', right_on='weighted_payment_date', how='outer')

# Plot the data using matplotlib
fig, ax1 = plt.subplots(figsize=(10, 6))

# Plot the receipt counts on the primary y-axis
sns.lineplot(x='receipt_date', y='receipt_count', data=monthly_counts, marker='o', ax=ax1, color='blue', label='Receipt Count')
ax1.set_xlabel('Date')
ax1.set_ylabel('Number of A/R Entries (Receipt)', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

# Create a secondary y-axis for the payment counts
ax2 = ax1.twinx()
sns.lineplot(x='weighted_payment_date', y='payment_count', data=monthly_counts, marker='o', ax=ax2, color='red', label='Payment Count')
ax2.set_ylabel('Number of A/R Entries (Payment)', color='red')
ax2.tick_params(axis='y', labelcolor='red')


# Disable the legend for the secondary y-axis
ax1.get_legend().remove()

# Combine legends
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc='upper left')

# Set the title and show the plot
plt.title('Monthly Count of A/R Entries by Receipt Date and Payment Date')
fig.tight_layout()
plt.grid(True)
plt.show()

In [23]:
# Remove all enntries before receipt_date 01.01.2022
df = df[(df['receipt_date'] >= '2022-01-01') & (df['receipt_date'] <= '2024-05-01')]

### Reminders Dataset:

Only after 2022 reminders consistent

In [ ]:
df_reminders = pd.read_csv('reminders_df.csv')

In [ ]:
df_reminders['reminder_date'] = pd.to_datetime(df_reminders['reminder_date'])

#remove reminder date before 2021
df_reminders = df_reminders[df_reminders['reminder_date'] >= '2021-01-01']

# Create copies of the dataframe
graph_reminders = df_reminders.copy()

graph_reminders.set_index('reminder_date', inplace=True)


# Resample the data to monthly frequency and count the number of rows for each month
monthly_reminder_counts = graph_reminders.resample('ME').size().reset_index(name='reminder_count')

# Plot the data using matplotlib
fig, ax1 = plt.subplots(figsize=(10, 6))

# Plot the receipt counts on the primary y-axis
sns.lineplot(x='reminder_date', y='reminder_count', data=monthly_reminder_counts, marker='o', ax=ax1, color='blue', label='Receipt Count')
ax1.set_xlabel('Date')
ax1.set_ylabel('Number of Reminders', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

# Combine legends
lines, labels = ax1.get_legend_handles_labels()

# Set the title and show the plot
plt.title('Monthly Count of Reminders')
fig.tight_layout()
plt.grid(True)
plt.show()

print("Total Reminders count: ", len(df_reminders))

### Clarifications Dataset

In [ ]:
df_clarifications = pd.read_csv('clarifications.csv')

In [ ]:

df_clarifications['created_at'] = pd.to_datetime(df_clarifications['created_at'])

# Remove created_at dates before 2021
df_clarifications = df_clarifications[df_clarifications['created_at'] >= '2021-01-01']

# Create copies of the dataframe
graph_clarifications = df_clarifications.copy()

graph_clarifications.set_index('created_at', inplace=True)

# Resample the data to monthly frequency and count the number of rows for each month
monthly_clarification_counts = graph_clarifications.resample('ME').size().reset_index(name='clarification_count')

# Plot the data using matplotlib
fig, ax1 = plt.subplots(figsize=(10, 6))

# Plot the clarification counts on the primary y-axis
sns.lineplot(x='created_at', y='clarification_count', data=monthly_clarification_counts, marker='o', ax=ax1, color='red', label='Clarification Count')
ax1.set_xlabel('Date')
ax1.set_ylabel('Number of Clarifications', color='red')
ax1.tick_params(axis='y', labelcolor='red')

# Combine legends
lines, labels = ax1.get_legend_handles_labels()

# Set the title and show the plot
plt.title('Monthly Count of Clarifications')
fig.tight_layout()
plt.grid(True)
plt.show()

print("Total clafirication count: ", len(df_clarifications))

## "Amount" Column & "Amount Euro column

The amount column is heavily skewed.

1. log.transform 
2. "Amount" < 1 is dropped
3. Remove 99th percentile

### "amount_euro": 

In [ ]:
# Check distribution of amount columns
print("Distribution before cleaning")
print(df['amount_euro'].describe().round(2))

# Drop all "amount" that are less than 1
df = df[df['amount_euro'] >= 1]


# Skewness of amount:
mean_amount = df['amount_euro'].mean()
median_amount = df['amount_euro'].median()
sd_amount = df['amount_euro'].std()

skew_amount = 3 * (mean_amount - median_amount) / sd_amount

print("-------------------")
print("Distribution after cleaning")
print(df['amount_euro'].describe().round(2))
print("-------------------")
print("Skewness of 'amount' column:", skew_amount)


# Log transform the 'amount' column
df['log_amount_eur'] = np.log(df['amount_euro'])

# Create a figure with two subplots
fig, axes = plt.subplots(1, 2, figsize=(20, 6))

# Plot the histogram of the "amount" column on the first subplot
sns.histplot(df['amount_euro'], bins=500, kde=False, ax=axes[0])
axes[0].set_xlabel('Amount Euro')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Histogram of Amount in Euros')
axes[0].grid(True)

# Plot the histogram of the log-transformed "amount" column on the second subplot
sns.histplot(df['log_amount_eur'], bins=150, kde=False, ax=axes[1])
axes[1].set_xlabel('Log Amount Euro')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Histogram of Log Amount in Euros')
axes[1].grid(True)

# Adjust layout and show the plot
plt.tight_layout()
plt.show()

In [ ]:
## Normal probability plot for log_transfromed amount column

import statsmodels.api as sm

# Extract the log-transformed 'amount' column
log_amount_eur = df['log_amount_eur']

#normalize
log_amount_eur = (log_amount_eur - log_amount_eur.mean()) / log_amount_eur.std()

# Normal probability plot
sm.qqplot(log_amount_eur, line='45')
plt.title('Normal Probability Plot for log_amount_eur')
plt.xlabel('Theoretical Quantiles')
plt.ylabel('Sample Quantiles')
plt.grid(True)
plt.show()

## "Payment_terms_column"

Payment terms can be treated as a continuous feature, because although it is a discrete variable, it has sufficient cardinality.


Remove all payment terms >120 for consistency with DSO column

In [ ]:
#Print payment terms before removing outliers
print(df["payment_terms"].describe())

#number of unique payment terms
n_unique_payment_terms = df["payment_terms"].nunique()

df_payment_terms = df[df["payment_terms"] <= 120]
n_unique_payment_terms_2 = df_payment_terms["payment_terms"].nunique()

# Create a figure with two subplots
fig, axes = plt.subplots(1, 2, figsize=(20, 6))

sns.histplot(df['payment_terms'], bins=n_unique_payment_terms, kde=False, ax=axes[0])
axes[0].set_xlabel('Payment_terms')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Histogram of Payment_Terms')
axes[0].grid(True)

sns.histplot(df_payment_terms['payment_terms'], bins=n_unique_payment_terms_2, kde=False, ax=axes[1])
axes[1].set_xlabel('payment_terms')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Histogram of Payment_Terms <= 120')
axes[1].grid(True)


df = df[df["payment_terms"] <= 120]

#print payment term after removing outliers
print(df["payment_terms"].describe())

## Creation of dso

"dso" = weighted_payment_date - receipt_date

"dso" is our independent variable, the variable we want to predict


In [ ]:
# New variable "dso"
df['dso'] = (df['weighted_payment_date'] - df['receipt_date']).dt.days

df.dtypes

## Cleaning af dso

- Remove Entries with DSO >120, since defined as outliers
- Plot with and without outliers

In [ ]:
# Create a figure with two subplots
fig, axes = plt.subplots(1, 2, figsize=(20, 6))

# Plot the histogram of the "dso" column on the first subplot
sns.histplot(df['dso'], bins=100, kde=False, ax=axes[0])
axes[0].set_xlabel('DSO', fontsize=18)
axes[0].set_ylabel('Frequency', fontsize=18)
axes[0].set_title('Histogram of DSO', fontsize=20)
axes[0].tick_params(axis='both', which='major', labelsize=14)
axes[0].grid(True)

# Calculate mean and median
mean_dso = df['dso'].mean()
median_dso = df['dso'].median()

# Add vertical lines for mean and median
axes[0].axvline(mean_dso, color='r', linestyle='--', linewidth=2, label=f'Mean: {mean_dso:.2f}')
axes[0].axvline(median_dso, color='g', linestyle='-', linewidth=2, label=f'Median: {median_dso:.2f}')
axes[0].legend(fontsize=14)

# Check distribution of dso columns
summary_statistics = df['dso'].describe().round(2)
print("Summary statistics before removing outliers:")
print(" ")
print(summary_statistics)
print("---------------------------------------------")
print(" ")
print("Summary Statistics after removing outliers:")
(" ")

# Remove outliers
df = df[df["dso"] <= 120]

# Plot the histogram of the "dso" column without outliers on the second subplot
sns.histplot(df['dso'], bins=100, kde=True, ax=axes[1])
axes[1].set_xlabel('DSO', fontsize=18)
axes[1].set_ylabel('Frequency', fontsize=18)
axes[1].set_title('Histogram of DSO without Outliers', fontsize=20)
axes[1].tick_params(axis='both', which='major', labelsize=14)
axes[1].grid(True)

# Calculate mean and median without outliers
mean_dso_no_outliers = df['dso'].mean()
median_dso_no_outliers = df['dso'].median()

# Add vertical lines for mean and median without outliers
axes[1].axvline(mean_dso_no_outliers, color='r', linestyle='--', linewidth=2, label=f'Mean: {mean_dso_no_outliers:.2f}')
axes[1].axvline(median_dso_no_outliers, color='g', linestyle='-', linewidth=2, label=f'Median: {median_dso_no_outliers:.2f}')
axes[1].legend(fontsize=14)

# Check distribution of dso columns
summary_statistics = df['dso'].describe().round(2)
print(summary_statistics)

plt.tight_layout()
plt.show()

Plot monthly DSO as time series

In [ ]:
# Group by month and calculate the average DSO for each month
df['month'] = df['receipt_date'].dt.to_period('M')
monthly_avg_dso = df.groupby('month')['dso'].mean().reset_index()

# Convert 'month' back to datetime for plotting
monthly_avg_dso['month'] = monthly_avg_dso['month'].dt.to_timestamp()

# Plot the time series graph
plt.figure(figsize=(10, 6))
plt.plot(monthly_avg_dso['month'], monthly_avg_dso['dso'], marker='o', linestyle='-')
plt.title('Average Monthly DSO over Time')
plt.xlabel('Month')
plt.ylabel('Average DSO (Days)')
plt.grid(True)
plt.xticks(rotation=45)

# Annotate each point with the month
for i, row in monthly_avg_dso.iterrows():
    plt.annotate(row['month'].strftime('%m'), (row['month'], row['dso']), textcoords="offset points", xytext=(0,10), ha='center', rotation=45)

plt.tight_layout()
plt.show()

## Creation of dbt

In [ ]:
df['dbt'] = (df['weighted_payment_date'] - df['due_date']).dt.days

# Create a figure with two subplots
fig, axes = plt.subplots(1, 2, figsize=(20, 6))

# Plot the histogram of the "amount" column on the first subplot
sns.histplot(df['dbt'], bins=150, kde=False, ax=axes[0])
axes[0].set_xlabel('dbt')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Histogram of dbt')
axes[0].grid(True)

# Check distribution of dso columns
summary_statistics = df['dbt'].describe().round(2)
print("Summary statistics before removing outliers:")
print(" ")
print(summary_statistics)
print("---------------------------------------------")
print(" ")
print("Summary Statistics after removing outliers:")
(" ")




# Plot the histogram of the log-transformed "amount" column on the second subplot
sns.histplot(df['dbt'], bins=120, kde=False, ax=axes[1])
axes[1].set_xlabel('DBT')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Histogram of DBT without Outliers')
axes[1].grid(True)



# Check distribution of dso columns
summary_statistics = df['dbt'].describe().round(2)
print(summary_statistics)

## Plotting distribution of columns in raw dataset

In [ ]:
# get list of columns
columns = df.columns

# Define the number of rows and columns of grid of subplots
nrows = 5
ncols = 3

# Create subplots
fig, axes = plt.subplots(nrows, ncols, figsize=(20, 18))

# Flatten the axes array for easy iteration
axes = axes.flatten()

# Iterate over the columns and plot histograms
for j in range(len(columns)):
    column = columns[j]
    sns.histplot(df[column], bins=100, kde=False, ax=axes[j])
    axes[j].set_title(f'Histogram of {column}')
    axes[j].set_xlabel(column)
    axes[j].set_ylabel('Frequency')
    axes[j].grid(True)

# Hide any unused subplots
for j in range(len(columns), len(axes)):
    fig.delaxes(axes[j])

# Display the plot
plt.tight_layout()
plt.show()